# 데이터 로드

In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from dotenv import load_dotenv
import os
import urllib.request
from datetime import datetime
import re
import urllib.parse

load_dotenv()
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')

def getnews_data(word, display=100):
    
    encoded_query = urllib.parse.quote(word)  # 한글을 URL 인코딩
    url = 'https://openapi.naver.com/v1/search/news.json?query={}&display={}'.format(encoded_query, display)
    
    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            items = response.json()['items']
            news_list = []
            
            for idx, item in enumerate(items):
                # HTML 태그 제거
                link = item.get('link')
                title = re.sub('<.*?>', '', item.get('title', ''))
                description = re.sub('<.*?>', '', item.get('description', ''))
                
                news_data = {
                    'no': idx + 1,
                    'title': title, #뉴스 제목
                    'link': item.get('link', ''), #뉴스 링크
                    'description': description, #뉴스 요약
                    'pubDate': item.get('pubDate', ''), #발행 날짜
                    'collected_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S") #데이터 수집 기간
                }
                news_list.append(news_data)
                
                # 진행상황 출력
                #print(f"{idx + 1}. {title}")
            
            print(f"'{word}' 관련 뉴스 {len(news_list)}개 수집 완료")
            news_df = pd.DataFrame(news_list)
            return news_df
            
        else:
            print(f"API 오류: {response.status_code}")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"오류 발생: {e}")
        return pd.DataFrame()

In [44]:
getnews_data('서울날씨', 100)



'서울날씨' 관련 뉴스 100개 수집 완료


no                                              title  \
0     1                                        [신간] 1도의 가격   
1     2                       서울 오후 흐림, 현재 기온 25.6도…습도 75%   
2     3        [현장] 서울로보틱스, 새로운 방식의 자율주행으로 글로벌 시장 공략 '박...   
3     4                  [날씨] 내일∼모레 서쪽 호우...중부·전북 강하고 많은 비   
4     5              [내일날씨] 서울 오전 23도 흐리고 가끔 비…오후 28도까지 올라   
..  ...                                                ...   
95   96                  수박, 전통시장서 평균 3만원 돌파…열흘새 5000원 올랐다   
96   97                      [날씨] 전국 흐리고 비…내일부터 서쪽 중심 많은 비   
97   98                       [경기·인천 날씨] 흐리고 곳곳 비…최고온도 29도   
98   99  '컴백' 윤산하 &quot;팬들 실망 안 시킬 무대, 자신 있어요&quot;[EN:터뷰]   
99  100                    이른 폭염, 폭우 습격…오락가락 날씨에 농사짓기 ‘고역’   

                                                 link  \
0   https://n.news.naver.com/mnews/article/366/000...   
1   https://www.joongangenews.com/news/articleView...   
2   https://www.joongangenews.com/news/articleView...   
3   https://n.news.naver.com/mnews/article/052/000...   
4   https://www.joongangenews.com/news/articleView...   
..                                                ...   
95  https://n.news.naver.com/mnews/article/016/000...   
96  https://n.news.naver.com/mnews/article/422/000...   
97  https://www.kgnews.co.kr/news/article.html?no=...   
98  https://n.news.naver.com/mnews/article/079/000...   
99  https://n.news.naver.com/mnews/article/662/000...   

                                          description  \
0   “지금도 0.1도의 변화가 일상의 모든 가격을 결정하고 있다” 7월 초 기상 관측 ...   
1   |중앙이코노미뉴스 이상민 기자|출처=한국관광공사  7월 15일 오후 기준 서울특별시...   
2   이 대표는 &quot;서울로보틱스는 지난 7년간 독일의 폭설, 미국 텍사스의 흙먼지...   
3   서울은 비가 그치고 구름만 다소 지나고 있습니다. 오늘은 이렇게 소강상태가 이어지거...   
4   |중앙이코노미뉴스 이상민 기자|출처=한국관광공사 내일인 7월 16일에 서울특별시 최...   
..                                                ...   
95  서울 송파구 가락동농수산물도매시장에서 시민들이 수박을 고르고 있다. [연합] 수박 ...   
96  자세한 날씨는 기상캐스터 연결해서 알아보겠습니다. 김민지 캐스터. [캐스터] 오늘도...   
97  이어 ▲인천 22~28도, ▲강화 21~27도, ▲백령도 22~28도, ▲서울 21...   
98  지난 3일 서울 강남구의 한 카페에서 열린 라운드 인터뷰에서, 윤산하는 새 앨범 준...   
99  오락가락하는 날씨 탓에 농가들의 시름만 깊어지고 있다. ◆뜨거우나 무더우나…힘겨운 ...   

                            pubDate       collected_time  
0   Tue, 15 Jul 2025 15:32:00 +0900  2025-07-15 15:39:50  
1   Tue, 15 Jul 2025 15:30:00 +0900  2025-07-15 15:39:50  
2   Tue, 15 Jul 2025 15:28:00 +0900  2025-07-15 15:39:50  
3   Tue, 15 Jul 2025 15:24:00 +0900  2025-07-15 15:39:50  
4   Tue, 15 Jul 2025 15:20:00 +0900  2025-07-15 15:39:50  
..                              ...                  ...  
95  Tue, 15 Jul 2025 08:18:00 +0900  2025-07-15 15:39:50  
96  Tue, 15 Jul 2025 08:10:00 +0900  2025-07-15 15:39:50  
97  Tue, 15 Jul 2025 08:10:00 +0900  2025-07-15 15:39:50  
98  Tue, 15 Jul 2025 08:02:00 +0900  2025-07-15 15:39:50  
99  Tue, 15 Jul 2025 08:01:00 +0900  2025-07-15 15:39:50  

[100 rows x 6 columns]

# 중복 제거 & 데이터 저장(excel) & 업데이트

In [52]:
from pathlib import Path
from datetime import datetime
def update_and_save_news(word, display=100, file_path=None):
    #1. 파일 경로 설정
    if file_path is None:
        file_path = f"news_data/{word}_news.xlsx"

    print(f"=== '{word}' 뉴스 업데이트 시작 ===")
    print(f"수집 개수: {display}개")
    print(f"저장 경로: {file_path}")
    print("-" * 50)

    #2. 기존 데이터 로드
    existing_df = pd.DataFrame()
    if Path(file_path).exists():
        try:
            existing_df = pd.read_excel(file_path)
            print(f"기존 데이터 로드 완료: {len(existing_df)} 개")
        except Exception as e:
            print(f"기존 데이터 로드 오류 : {e}")
    else:
        print(f"기존 데이터 로드 실패 : {file_path} 파일이 존재하지 않음. 새로 생성 요망")

    #3. 새로운 뉴스 데이터 수집
    new_news_df = getnews_data(word, display)
    if new_news_df.empty:
        print("새로운 데이터 수집에 실패했습니다.")
        return False
    
    # 4. 중복 제거
    if existing_df.empty:
        new_df_filtered = new_news_df
        duplicates_count = 0
    else:
        new_df_filtered = new_news_df[~new_news_df['link'].isin(existing_df['link'])]
        duplicates_count = len(new_news_df) - len(new_df_filtered)
    
    print(f"중복 제거: {duplicates_count}개")
    print(f"새로운 뉴스: {len(new_df_filtered)}개")
    
    # 5. 새로운 뉴스가 없으면 종료
    if new_df_filtered.empty:
        print("새로운 뉴스가 없습니다.")
        return False
    
    # 6. 데이터 병합 (새로운 뉴스를 맨 위에 추가)
    if existing_df.empty:
        final_df = new_df_filtered
    else:
        final_df = pd.concat([new_df_filtered, existing_df], ignore_index=True)
    
    # 7. no 컬럼 재정렬
    final_df['no'] = range(1, len(final_df) + 1)
    
    # 8. 엑셀 파일로 저장
    try:
        final_df.to_excel(file_path, index=True)
        
        print(f"'{word}' 뉴스 데이터가 '{file_path}'에 저장되었습니다.")
        print(f"새로 추가된 뉴스: {len(new_df_filtered)}개")
        print(f"전체 뉴스 개수: {len(final_df)}개")
        
        return True
        
    except Exception as e:
        print(f"파일 저장 오류: {e}")
        return False

    

In [53]:
update_and_save_news('서울날씨', 100)

=== '서울날씨' 뉴스 업데이트 시작 ===
수집 개수: 100개
저장 경로: news_data/서울날씨_news.xlsx
--------------------------------------------------
기존 데이터 로드 실패 : news_data/서울날씨_news.xlsx 파일이 존재하지 않음. 새로 생성 요망
'서울날씨' 관련 뉴스 100개 수집 완료
중복 제거: 0개
새로운 뉴스: 100개
파일 저장 오류: Cannot save file into a non-existent directory: 'news_data'


False